In [7]:
import pandas as pd
from experta import *

# 🔹 Load dataset (optional, if using existing data)
file_path = r"C:\Users\mrahm\Downloads\ANU projects\Heart_Disease_Detection\data\cleaned_data.csv"
df = pd.read_csv(file_path) if file_path else None

# 🎯 Define the Expert System
class HeartDiseaseExpert(KnowledgeEngine):
    
    # 🔹 Rule: High Cholesterol & High BP → High Risk
    @Rule(Fact(chol=P(lambda x: x > 240)) & Fact(trestbps=P(lambda x: x > 140)))
    def high_cholesterol_bp(self):
        print("⚠ High Risk: High cholesterol and high blood pressure detected!")

    # 🔹 Rule: Chest Pain Type (cp) severe → High Risk
    @Rule(Fact(cp=P(lambda x: x >= 2)))
    def severe_chest_pain(self):
        print("⚠ High Risk: Severe chest pain detected!")

    # 🔹 Rule: High Resting Blood Pressure
    @Rule(Fact(trestbps=P(lambda x: x > 130)))
    def high_bp_warning(self):
        print("⚠ Moderate Risk: Elevated resting blood pressure!")

    # 🔹 Rule: High Fasting Blood Sugar
    @Rule(Fact(fbs=1))
    def high_fbs(self):
        print("⚠ Warning: High fasting blood sugar detected!")

    # 🔹 Rule: High Heart Rate
    @Rule(Fact(thalach=P(lambda x: x > 160)))
    def high_heart_rate(self):
        print("⚠ Warning: Unusually high heart rate!")

    # 🔹 Rule: Exercise-Induced Angina (exang)
    @Rule(Fact(exang=1))
    def exercise_angina(self):
        print("⚠ Moderate Risk: Exercise-induced angina detected!")

    # 🔹 Rule: ST Depression (oldpeak) > 2.0
    @Rule(Fact(oldpeak=P(lambda x: x > 2.0)))
    def high_oldpeak(self):
        print("⚠ Warning: Significant ST depression detected!")

    # 🔹 Rule: Slope of the Peak (downsloping)
    @Rule(Fact(slope=1))
    def downsloping_st(self):
        print("⚠ High Risk: Downsloping ST segment detected!")

    # 🔹 Rule: More than 1 major vessel affected
    @Rule(Fact(ca=P(lambda x: x > 1)))
    def multi_vessel_disease(self):
        print("⚠ High Risk: Multiple major vessels affected!")

    # 🔹 Rule: High-Risk Thalassemia
    @Rule(Fact(thal=3))
    def high_risk_thal(self):
        print("⚠ High Risk: High-risk Thalassemia detected!")

# 🎯 Function to get user input (if not using a dataset)
def get_user_input():
    print("\n🔍 Enter Patient's Medical Data:")
    return {
        "age": int(input("Age: ")),
        "sex": int(input("Sex (0 = Female, 1 = Male): ")),
        "cp": int(input("Chest Pain Type (0-3): ")),
        "trestbps": int(input("Resting Blood Pressure: ")),
        "chol": int(input("Cholesterol Level: ")),
        "fbs": int(input("Fasting Blood Sugar (0 = No, 1 = Yes): ")),
        "restecg": int(input("Resting ECG Result (0-2): ")),
        "thalach": int(input("Max Heart Rate: ")),
        "exang": int(input("Exercise-Induced Angina (0 = No, 1 = Yes): ")),
        "oldpeak": float(input("ST Depression (oldpeak): ")),
        "slope": int(input("Slope of ST (0-2): ")),
        "ca": int(input("Number of Major Vessels (0-3): ")),
        "thal": int(input("Thalassemia (0-3): "))
    }

# 🎯 Function to run the expert system
def run_expert_system(data=None):
    engine = HeartDiseaseExpert()
    engine.reset()
    
    # 🟢 If data is provided (e.g., from dataset)
    if data:
        for key, value in data.items():
            engine.declare(Fact(**{key: value}))
    else:
        # 🔹 Otherwise, take user input
        user_data = get_user_input()
        for key, value in user_data.items():
            engine.declare(Fact(**{key: value}))

    print("\n🩺 **Health Risk Assessment Results:**")
    print("------------------------------------")
    engine.run()

# ✅ Run the system for user input
if __name__ == "__main__":
    run_expert_system()



🔍 Enter Patient's Medical Data:


Age:  50
Sex (0 = Female, 1 = Male):  1
Chest Pain Type (0-3):  2
Resting Blood Pressure:  150
Cholesterol Level:  180
Fasting Blood Sugar (0 = No, 1 = Yes):  1
Resting ECG Result (0-2):  2
Max Heart Rate:  140
Exercise-Induced Angina (0 = No, 1 = Yes):  1
ST Depression (oldpeak):  32
Slope of ST (0-2):  2
Number of Major Vessels (0-3):  2
Thalassemia (0-3):  1



🩺 **Health Risk Assessment Results:**
------------------------------------
⚠ High Risk: Multiple major vessels affected!
⚠ Warning: Significant ST depression detected!
⚠ Moderate Risk: Exercise-induced angina detected!
⚠ Warning: High fasting blood sugar detected!
⚠ Moderate Risk: Elevated resting blood pressure!
⚠ High Risk: Severe chest pain detected!


In [21]:
import pandas as pd
import joblib
from experta import KnowledgeEngine
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

validation_data_path = r"C:\Users\mrahm\Downloads\ANU projects\Heart_Disease_Detection\data\cleaned_data.csv"
df_validation = pd.read_csv(validation_data_path)

model_path = r"C:\Users\mrahm\Downloads\ANU projects\Heart_Disease_Detection\heart_disease_model.pkl"
decision_tree_model = joblib.load(model_path)

# ✅ تحقق من البيانات
if df_validation.empty:
    print(" No validation data available.")
else:
    print(" Validation Data Loaded Successfully!")
    
    X_val = df_validation.drop(columns=["target"])
    y_val = df_validation["target"]

    y_pred_tree = decision_tree_model.predict(X_val)

    acc_tree = accuracy_score(y_val, y_pred_tree)
    prec_tree = precision_score(y_val, y_pred_tree, zero_division=1)
    rec_tree = recall_score(y_val, y_pred_tree, zero_division=1)
    f1_tree = f1_score(y_val, y_pred_tree, zero_division=1)

    print("\n🌳 Decision Tree Model Performance:")
    print(f"🎯 Accuracy: {acc_tree:.2f}")
    print(f"🎯 Precision: {prec_tree:.2f}")
    print(f"🎯 Recall: {rec_tree:.2f}")
    print(f"🎯 F1-score: {f1_tree:.2f}")

    # 🔹 تشغيل النظام الخبير
    class HeartDiseaseExpert(KnowledgeEngine):
        def __init__(self):
            super().__init__()
            self.predictions = []

        def predict(self, patient_data):
            self.predictions = []  # مسح التنبؤات السابقة
            self.reset()
            for key, value in patient_data.items():
                self.declare(Fact(**{key: value}))
            self.run()
            return self.predictions[0] if self.predictions else 0  # إذا لم تكن هناك تنبؤات، فالإفتراض 0

    expert_system = HeartDiseaseExpert()
    
    y_pred_expert = [expert_system.predict(row.to_dict()) for _, row in X_val.iterrows()]

    acc_expert = accuracy_score(y_val, y_pred_expert)
    prec_expert = precision_score(y_val, y_pred_expert, zero_division=1)
    rec_expert = recall_score(y_val, y_pred_expert, zero_division=1)
    f1_expert = f1_score(y_val, y_pred_expert, zero_division=1)

    print("\n🤖 Expert System Performance:")
    print(f"🎯 Accuracy: {acc_expert:.2f}")
    print(f"🎯 Precision: {prec_expert:.2f}")
    print(f"🎯 Recall: {rec_expert:.2f}")
    print(f"🎯 F1-score: {f1_expert:.2f}")

    print("\n📌 Comparison of Decision Making:")
    print("Decision Tree: Data-driven, statistical learning model.")
    print("Expert System: Human-defined rules, explicit logic.")


 Validation Data Loaded Successfully!

🌳 Decision Tree Model Performance:
🎯 Accuracy: 0.97
🎯 Precision: 0.96
🎯 Recall: 0.98
🎯 F1-score: 0.97

🤖 Expert System Performance:
🎯 Accuracy: 0.49
🎯 Precision: 1.00
🎯 Recall: 0.00
🎯 F1-score: 0.00

📌 Comparison of Decision Making:
Decision Tree: Data-driven, statistical learning model.
Expert System: Human-defined rules, explicit logic.
